<a href="https://colab.research.google.com/github/onleey/Machine-Learning_demo/blob/master/651_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/001'

Mounted at /content/drive
/content/drive/MyDrive/001


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.2 MB/s eta 0:00:00


1_3 Word2Vec
- Word2Vec은 분산 표현 형태의 단어 임베딩 모델이다.
- Word2Vec은 2013년 구글에서 발표했으며 가장 많이 사용하고 있는 단어 임베딩 모델이다.
- Word2Vec 모델은 CBOW(continuous bag-of-words)와 skip-gram 두가지 모델이다.
- CBOW 모델 맥락(context word)이라 표현되는 주변 단어들을 이용해 타깃단어를 예측하는 신경망 모델이다. 신경망의 입력을 주변 단어들로 구성하고 출력을 타깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용한다.
- skp-gram모델은 CBOW 모델과 반대로 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델이다. 입출력이 CBOW모델과 반대로 되어 있기 때문에 skip-gram모델이 CBOW 모델에 비해 예측해야 하는 맥락이 많아진다. 따라서, 단어 분산 표현력이 우수해 CBOW 모델에 비해 임베딩 품질이 우수하다. 반면 CBOW모델은 타깃 단어의 손실만 계산하면 되기 때문에 학습 속도가 빠른 장점이 있다.
- CBOW 모델에서는 타깃 단어를 예측하기 위해 앞뒤 단어를 확인할 수 있다. 이때 앞뒤로 몇개의 단어까지 확인할지 결정할 수 있는데 이 범위를 윈도우(window)라고한다. 예를 들어 윈도우 크기를 2로 한 경우 타깃 단어의 앞뒤 단어 2개까지 모델 학습을 위해 사용된다.
- Word2Vec의 단어 임베딩은 해당 단어를 밀집 벡터로 표현하며 학습을 통해 의미상 비슷한 단어들을 비슷한 벡터 공간에 위치시킨다. 또한 벡터 특성상 의미에 따라 방향을 갖게 된다.
- 임베딩된 벡터들 간 연산이 가능하기 때문에 단어 간 관계를 계산할 수 있다. '왕'과'여왕'의 방향 차이만큼 '남자'와'여자'의 방향 차이가 생긴다.

Word2Vec

In [8]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data


# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./datasets/ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)
print(docs[:5])

# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
# vector_size: 단어 임베딩 벡터의 차원(크기), window: 주변 단어 윈도우의 크기
# hs:1(모델학습에 softmax 사용), 0(negative 옵션 값이 0이 아닐때 음수 샘플링으로 사용)
# min_count: 단어 최소 빈도 수 제한(설정된 min_count  빈도 수 이하의 단어들은 학습하지 않음)
# sg: 0(CBOW 모델), 1(skip-gram 모델)
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('./datasets/nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)



1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  0.4437077045440674
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  100.81273245811462
[['때'], ['디자인', '학생', '외국', '디자이너', '전통', '발전', '문화', '산업', '사실', '우리나라', '시절', '끝', '열정', '노라', '노', '전통', '사람', '꿈', '수', '것', '감사'], ['폴리스', '스토리', '시리즈', '뉴', '최고'], ['연기', '것', '라고', '생각', '몰입', '영', '화지'], ['안개', '밤하늘', '초승달', '영화']]
3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  116.03443121910095
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  116.08982610702515
corpus_count :  200000
corpus_total_words :  1076896


In [21]:
print(dir(model))
#print(model.sorted_vocab)
print(dir(model.wv))
print(model.wv.key_to_index)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__ignoreds', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__numpys', '__recursive_saveloads', '__reduce__', '__reduce_ex__', '__repr__', '__scipys', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_check_corpus_sanity', '_check_training_sanity', '_clear_post_train', '_do_train_epoch', '_do_train_job', '_get_next_alpha', '_get_thread_working_mem', '_job_producer', '_load_specials', '_log_epoch_end', '_log_epoch_progress', '_log_progress', '_log_train_end', '_raw_word_count', '_save_specials', '_scan_vocab', '_smart_save', '_train_epoch', '_train_epoch_corpusfile', '_worker_loop', '_worker_loop_corpusfile', 'add_lifecycle_event', 'add_null_word', 'alpha', 'batch_words', 'build_vocab', 'build_vocab_from_freq', 'cbow_mean', 'comment', 'compute_l

word2vec 모델활용
위에서 생성된 word2vec 모델 파일을 읽어와 실제로 단어 임베딩된 값과 벡터 공간상의 유사한

In [15]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('./datasets/nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print(len(model.wv['사랑']))
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
# model.wv.similarity( ) : 두 단어 간의 유사도를 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))





corpus_total_words :  1076896
200
사랑 :  [ 0.13843498 -0.49585885  0.14830585  0.25001565 -0.2748962  -0.00585074
  0.08460066  0.08610283 -0.10643353  0.3778886  -0.20350738  0.02804426
  0.02542877 -0.14179623 -0.10187252  0.42765757 -0.14788139 -0.08133226
 -0.37628254 -0.536132    0.41459668  0.3325955  -0.1660681   0.23424388
 -0.04700854  0.05539669  0.12625033 -0.25673857 -0.03028387 -0.2574538
  0.07273091  0.1669274   0.17022878  0.03830987  0.1822559   0.38572606
  0.08623144  0.00329793 -0.38677368 -0.23187847 -0.27022186  0.06465048
 -0.0740785  -0.39591718  0.5344503   0.42093042 -0.11665963 -0.2385033
  0.3343829   0.28298464  0.17170186 -0.25508782  0.05214195  0.01519619
 -0.09695343 -0.22314972  0.08411141 -0.32787222 -0.12242746  0.16529639
  0.13663363  0.04216238 -0.35223284  0.09680844 -0.24172123 -0.13681802
 -0.15852895  0.1274234  -0.08413866  0.6800417   0.05495577 -0.16882396
  0.2570553   0.08692737  0.25643334 -0.29162234  0.2159014  -0.34437612
 -0.20648502 

In [20]:
# 가장 유사한 단어 추출
# model.wv.most_similar( ) : 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환한다.
# 즉, 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환한다. topn인자는 반환되는 유사한 단어 수를 의미하며,
# 예제에서는 5개까지 유사한 단어를 반환한다.

print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))
print(model.wv.most_similar(["안성기","최고"], topn=5))

# 안성기와 최고와 밀접한 증거들 중에  조재현이 방해가 되니까 조재현 제외

print(model.wv.most_similar(["안성기","최고"],negative=['조재현'], topn=5))


[('김승우', 0.9359989166259766), ('최민수', 0.9353854060173035), ('박신양', 0.9341099858283997), ('양동근', 0.9335288405418396), ('정재영', 0.9331846237182617)]
[('엑스맨', 0.8010914325714111), ('포터', 0.7984391450881958), ('스타워즈', 0.7896637320518494), ('반지의 제왕', 0.7862001061439514), ('해리', 0.7842392921447754)]
[('최상', 0.8719119429588318), ('우수', 0.8685317039489746), ('OST', 0.8683410286903381), ('이범수', 0.8657281994819641), ('진국', 0.8649217486381531)]
[('한국영', 0.7173784375190735), ('과언', 0.6997277736663818), ('단언', 0.6927541494369507), ('정점', 0.6849888563156128), ('화중', 0.684207022190094)]
